In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-22 02:11:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.73MB/s    in 0.2s    

2021-11-22 02:11:32 (4.73 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [5]:
print((df.count(), len(df.columns)))

(4864249, 15)


## Drop duplicates and incomplete rows

In [6]:
df = df.drop_duplicates()
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43750961|R10010KWXLSPF3|B004UUHX9M|     247428181|Star Wars Deluxe ...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|My 5-year old lov...| 2014-12-05|
|         US|   37257103|R100BG8WSTAYPG|B00ENKGLHO|       9108602|Ducati 1199 Panig...|            Toys|          5|    

In [7]:
print((df.count(), len(df.columns)))

(4864249, 15)


In [8]:
df = df.dropna()
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43750961|R10010KWXLSPF3|B004UUHX9M|     247428181|Star Wars Deluxe ...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|My 5-year old lov...| 2014-12-05|
|         US|   37257103|R100BG8WSTAYPG|B00ENKGLHO|       9108602|Ducati 1199 Panig...|            Toys|          5|    

In [9]:
print((df.count(), len(df.columns)))

(4863497, 15)


## Examine the schema

In [10]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



review_date column is a string and needs to be changed to date format before load

## Create a new DataFrames for load process

In [11]:
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date

# Create the review_id DataFrame. 
# Convert the 'review_date' column to a date datatype
review_id = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010KWXLSPF3|   43750961|B004UUHX9M|     247428181| 2014-12-05|
|R100BG8WSTAYPG|   37257103|B00ENKGLHO|       9108602| 2014-04-29|
|R100F00UQBXLAL|   14056891|B004O3CSX6|     579644974| 2011-11-11|
|R1010FUSHO8ASL|   30419449|B000VXNYQI|     496402705| 2012-12-17|
|R101LBRJ366XL1|   36812928|B000JFJC4G|     870047482| 2015-02-28|
|R101LKRSNW6OIK|   28394801|B000FPCWXI|     841597184| 2014-07-06|
|R101TEA798DC3M|   19838846|B0017VO5AW|     484336160| 2015-06-14|
|R101UKS45S1Z0O|   18589384|B000GHWTS8|     632533309| 2009-12-12|
|R101Y71FKFPSEO|   17270128|B004U7JJWE|     366785521| 2013-03-05|
|R102ETLR4XJZFM|   37637960|B000S94NWE|     327618371| 2013-11-08|
|R102IJ4R9UE6QH|   21873089|B00HTMCF0U|     841568092| 2015-02-16|
|R1038ZA60A3YYJ|    2395887|B004S8F7QM|     829220659| 2014-11

In [12]:
# Create the products DataFrame and drop duplicates to contain only unique values. 
products = df.select(["product_id", "product_title"]).drop_duplicates()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IANTUPU|Lego, Star Wars, ...|
|B002TMK962|Rock - Concert Ti...|
|B00AOTTSQY|Adult & Children ...|
|B00CH3O5S6|48 Wholesale Wood...|
|B00MF3SAF2|Teenitor 3.7V 680...|
|B004D0C5WY|ThinkMax Silver B...|
|B00NHQHJZS|LEGO Technic Arct...|
|B00NQJ2DBG|Super Bright Long...|
|B0082IGA06|Yu-Gi-Oh! - Dark ...|
|1609581520|American Girl Car...|
|B004RGYVLC|DC Comics POP Her...|
|B0053EX3D4|Inuyasha Kirara T...|
|B00OTLHNI8|Bloco Toys T-Rex ...|
|B00BXEYI8W|Kid Galaxy Robot ...|
|B00JUG2NSM|Weiss Schwarz Kil...|
|B00LVPY252|Maxx Action 30" T...|
|B00IGRHWSC|Mattel Hot Wheels...|
|B00U6FU3YM|Educational Insig...|
|B00Z7FS3Q6|Beach Ball Set, 3...|
|B000WEAGTY|Inflatable Tradit...|
+----------+--------------------+
only showing top 20 rows



In [13]:
# Create the customers DataFrame and rename column to customer_count
customers = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40305706|             1|
|    3887504|             4|
|   41739251|             1|
|   28952249|            51|
|   34724923|             1|
|   14225559|             3|
|   26285304|             2|
|     167919|             1|
|   26815475|             9|
|   14908291|             6|
|   27672367|             2|
|   24219298|             4|
|   48356484|            27|
|   18461577|             4|
|   46086404|             1|
|   49996826|             7|
|   20159319|             7|
|   24353265|             2|
|   19833133|             4|
|   51567618|             2|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the vine DataFrame
vine = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10010KWXLSPF3|          5|            0|          0|   N|
|R100BG8WSTAYPG|          5|            1|          1|   N|
|R100F00UQBXLAL|          5|            0|          0|   N|
|R1010FUSHO8ASL|          5|            0|          1|   N|
|R101LBRJ366XL1|          5|            0|          0|   N|
|R101LKRSNW6OIK|          5|            0|          0|   N|
|R101TEA798DC3M|          4|            1|          1|   N|
|R101UKS45S1Z0O|          5|            0|          0|   N|
|R101Y71FKFPSEO|          5|            0|          0|   N|
|R102ETLR4XJZFM|          5|            0|          0|   N|
|R102IJ4R9UE6QH|          5|            0|          0|   N|
|R1038ZA60A3YYJ|          5|            0|          0|   N|
|R1039LT1Z5D6A1|          4|            0|          0|   N|
| R103HL2RILQNH|          5|           1

## Write DataFrame to RDS

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-01.cvudwnrntrve.us-west-2.rds.amazonaws.com:5432/my_data_class_db_alfred"
config = {"user":"my_data_class_db_alfred", 
          "password": "AhZleN58!D", 
          "driver":"org.postgresql.Driver"}

In [19]:
# Write review_id to table in RDS
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [20]:
# Write products to table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [21]:
# Write customers to table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [22]:
# Write vine to table in RDS
vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)